# CONSTANTS USED FOR GENERATION. FEEL FREE TO EDIT.
To edit outputs, scroll down until you find the next massive header.

In [1]:
# What types of areas different rooms can be
AREA_TYPES = ["desk", "room", "building"]
# The types of features rooms can include
FEATURES = ["Screen", "Camera", "Microphone"]

# How many users to generate
USER_COUNT = 0
# How many areas to generate
AREA_COUNT = 20
# How many bookings to generate. Note that these may be silently
# dropped if they overlap with existsing bookings.
MAX_BOOKING_COUNT = 10000

***
Code below
***

In [6]:
import random
import string
from datetime import datetime, time, date, timedelta

def rand_text(length: int=None, whitespace: bool = False) -> str:
	text = string.ascii_letters
	if whitespace:
		text = text + " "
	if (not length):
		length = random.randint(3, 20)
	text = "".join(random.choices(text, k=length)).capitalize();
	return text

def rand_text_in(min: int = 5, max: int = 12, whitespace: bool = False) -> str:
	return rand_text(random.randint(min, max), whitespace)

def rand_bool(odds=0.5) -> bool:
	return random.random() < odds

def rand_timedelta(duration=128):
	return timedelta(
		hours=random.randint(0, duration - 1),
		minutes=random.randint(0, 59),
		seconds=random.randint(0, 59)
	)

def choices_x(arr: list, num: int = 3) -> list:
	arr = arr.copy()
	retval = []
	for _ in range(num):
		idx = random.randint(0, len(arr) - 1)
		retval.append(arr.pop(idx))
	return retval

In [3]:
import uuid
from uuid import UUID
class IdTicker:
	def get(self) -> uuid.UUID:
		newId = uuid.uuid4()

		return newId

In [4]:
import requests
def validate_response(response: requests.Response):
	if not response.ok:
	 response.raise_for_status()

In [5]:

SERVER = "http://localhost:8080"
SERVER_USER = SERVER + "/user"
SERVER_AREA = SERVER + "/area"
SERVER_BOOKING = SERVER + "/reservation"

# USER

In [6]:
class User:
	id: UUID
	firstName: str = "ERROR"
	lastName: str = "ERROR"
	email: str = "ERROR@ERROR.ERR"

	def __init__(self):
		self.firstName = rand_text()
		self.lastName = rand_text()
		self.email = rand_text() + "@" + rand_text() + "." + rand_text_in(2, 3)

	def get_json(self):
		return {
			"firstName": self.firstName,
			"lastName": self.lastName,
			"email": self.email
		}

In [7]:
for i in range(USER_COUNT):
	response = requests.post(SERVER_USER, json=User().get_json())
	validate_response(response)

In [7]:
def get_user_ids():
	users = requests.get(SERVER_USER).json()
	user_ids = []
	for user in users:
		user_ids.append(user.get("id"))
	print(f"Found {len(user_ids)} users")
	return user_ids
user_ids = get_user_ids()

Found 3500 users


# AREAS

In [9]:
def get_area_ids():
	areas_page = requests.get(SERVER_AREA + "?page=0").json()
	areas = areas_page.get("content")
	for page in range(1, areas_page.get("totalPages")):
		this_page = requests.get(SERVER_AREA + f"?page={page}").json()
		areas = [*areas, *this_page.get("content")]
	area_ids = []
	for area in areas:
		area_ids.append(area.get("id"))
	print(f"Found {len(area_ids)} areas")
	return area_ids
area_ids = get_area_ids()

Found 180 areas


In [10]:
class Area:
	administratorIds: list[UUID] = []
	superAreaId: UUID = None
	areaTypeId: str
	areaFeatureIds: list[str] = []
	capacity: int
	calendarLink: str
	name: str
	description: str
	reservable: bool = True

	def __init__(self):
		self.administratorIds = choices_x(user_ids)
		if rand_bool(0.2):
			self.superAreaId = random.choice(area_ids)
		self.areaTypeId = random.choice(AREA_TYPES)
		self.areaFeatureIds = choices_x(FEATURES, random.randint(1,3))
		self.capacity = random.randint(1,10)
		self.name = rand_text()
		self.description = rand_text_in(70, 150, whitespace=True)

	def get_json(self):
		return {
			"administratorIds": self.administratorIds,
			"superAreaId": self.superAreaId,
			"areaTypeId": self.areaTypeId,
			"areaFeatureIds": self.areaFeatureIds,
			"capacity": self.capacity,
			"name": self.name,
			"description": self.description,
			"reservable": self.reservable
		}

In [11]:
# Refresh area ids
area_ids = get_area_ids()
for i in range(AREA_COUNT):
	response = requests.post(SERVER_AREA, json=Area().get_json())
	validate_response(response)

Found 160 areas


# RESERVATIONS

In [14]:
class Reservation:
	user: UUID
	area: UUID
	startTime: datetime
	endTime: datetime
	comment: str

	def __init__(self):
		self.user = random.choice(user_ids)
		self.area = random.choice(area_ids)

		current_time = datetime.now()
		self.startTime = current_time + rand_timedelta()
		self.endTime = self.startTime + rand_timedelta(8)

		self.comment = rand_text_in(70, 150, whitespace=True)

	def get_json(self):
		return {
			"userId": self.user,
			"areaId": self.area,
			"startTime": self.startTime.isoformat(),
			"endTime": self.endTime.isoformat(),
			"comment": self.comment
		}

In [15]:
# Refresh IDs
user_ids = get_user_ids()
area_ids = get_area_ids()

count = 0

session = requests.Session()
for i in range(MAX_BOOKING_COUNT):
	response = session.post(SERVER_BOOKING, json=Reservation().get_json())
	if not response.ok and response.status_code != 409:
		response.raise_for_status()
	elif response.status_code != 409:
		count += 1;

f"Created {count} reservations"

Found 3500 users
Found 180 areas


'Created 10000 reservations'